## Align utterance in Memor to open subtitle parallel bitext

In [3]:
import json
import os

from lxml import etree
import lxml
from tqdm import tqdm
import pickle as pkl
import re
import json
import os

#### Load Friends Dataset

In [77]:
# Load Open Subtitle Bitext
xml_file = etree.parse('../open_subtitle/en-zh_cn.tmx')
# Collect subtitles
en_subtitle = []
zh_subtitle = []
root = xml_file.getroot()
count = 0
for node in tqdm(root.iter()):
    if node.tag=='seg':
        if count % 2 ==0:
            en_subtitle.append(node.text)
        else:
            zh_subtitle.append(node.text)
        count += 1

49845028it [00:58, 852163.01it/s] 


In [25]:
print(en_subtitle[0])

NameError: name 'en_subtitle' is not defined

In [22]:
with open('emory_nlp_dataset/json/friends_season_01.json') as f:
    data = json.load(f)

In [63]:

episodes = data['episodes']
print(type(episodes), len(episodes))
temp = episodes[0]
output = {}
for episode in episodes:
    transcripts = []
    episode_id = episode['episode_id']
    print(episode_id)
    scenes = episode['scenes']
    for scene in scenes:
        utterances = scene['utterances']
        for utt in utterances:
            transcript = utt['transcript'].strip()
            transcripts.append(transcript)
    output[episode_id] = transcripts

<class 'list'> 25
s03_e01
s03_e02
s03_e03
s03_e04
s03_e05
s03_e06
s03_e07
s03_e08
s03_e09
s03_e10
s03_e11
s03_e12
s03_e13
s03_e14
s03_e15
s03_e16
s03_e17
s03_e18
s03_e19
s03_e20
s03_e21
s03_e22
s03_e23
s03_e24
s03_e25


In [23]:
# Load Input Data
dir_path = 'emory_nlp_dataset/json/'
output = {}
for file_name in os.listdir(dir_path):
    # Load input data
    input_path = dir_path + file_name
    output_path = 'emory_nlp_dataset/pkl/friends_' + file_name.strip().split('_')[-1].split('.')[0] + '.pkl'
    with open(input_path) as f:
        data = json.load(f)
    episodes = data['episodes']
    print(type(episodes), len(episodes))
    temp = episodes[0]
    for episode in episodes:
        transcripts = []
        episode_id = episode['episode_id']
        print(episode_id)
        scenes = episode['scenes']
        for scene in scenes:
            utterances = scene['utterances']
            for utt in utterances:
                transcript = utt['transcript'].strip()
                transcripts.append(transcript)
        output[episode_id] = transcripts
data = output
# pkl.dump(" ", open('', 'wb'))

<class 'list'> 24
s01_e01
s01_e02
s01_e03
s01_e04
s01_e05
s01_e06
s01_e07
s01_e08
s01_e09
s01_e10
s01_e11
s01_e12
s01_e13
s01_e14
s01_e15
s01_e16
s01_e17
s01_e18
s01_e19
s01_e20
s01_e21
s01_e22
s01_e23
s01_e24
<class 'list'> 25
s06_e01
s06_e02
s06_e03
s06_e04
s06_e05
s06_e06
s06_e07
s06_e08
s06_e09
s06_e10
s06_e11
s06_e12
s06_e13
s06_e14
s06_e15
s06_e16
s06_e17
s06_e18
s06_e19
s06_e20
s06_e21
s06_e22
s06_e23
s06_e24
s06_e25
<class 'list'> 18
s10_e01
s10_e02
s10_e03
s10_e04
s10_e05
s10_e06
s10_e07
s10_e08
s10_e09
s10_e10
s10_e11
s10_e12
s10_e13
s10_e14
s10_e15
s10_e16
s10_e17
s10_e18
<class 'list'> 24
s07_e01
s07_e02
s07_e03
s07_e04
s07_e05
s07_e06
s07_e07
s07_e08
s07_e09
s07_e10
s07_e11
s07_e12
s07_e13
s07_e14
s07_e15
s07_e16
s07_e17
s07_e18
s07_e19
s07_e20
s07_e21
s07_e22
s07_e23
s07_e24
<class 'list'> 24
s04_e01
s04_e02
s04_e03
s04_e04
s04_e05
s04_e06
s04_e07
s04_e08
s04_e09
s04_e10
s04_e11
s04_e12
s04_e13
s04_e14
s04_e15
s04_e16
s04_e17
s04_e18
s04_e19
s04_e20
s04_e21
s04_e22
s04_e2

In [24]:
print(len(output.keys()))

236


In [72]:
print(data['s01_e02'])

["What you guys don't understand is, for us, kissing is as important as any part of it.", "Yeah, right!.......Y'serious?", 'Oh, yeah!', 'Everything you need to know is in that first kiss.', 'Absolutely.', "Yeah, I think for us, kissing is pretty much like an opening act, y'know? I mean it's like the stand-up comedian you have to sit through before Pink Floyd comes out.", "Yeah, and-and it's not that we don't like the comedian, it's that-that... that's not why we bought the ticket.", "The problem is, though, after the concert's over, no matter how great the show was, you girls are always looking for the comedian again, y'know? I mean, we're in the car, we're fighting traffic... basically just trying to stay awake.", "Yeah, well, word of advice: Bring back the comedian. Otherwise next time you're gonna find yourself sitting at home, listening to that album alone.", '....Are we still talking about sex?', "No, it's good, it is good, it's just that- mm- doesn't she seem a little angry?", 'W

In [73]:
def get_segments(episode_key, all_data):
    # Gather utteranes the defined episode
    segments = []
    pattern = r'\.|\,|\?|\!|\;'
    for utt in all_data[episode_key]:
        for item in re.split(pattern, utt):
                    tokens = item.strip().split(' ')
                    length = len(tokens)
                    num_iter = length // 6
                    for i in range(num_iter):
                        segments.append(" ".join(tokens[i*6: i*6+6]))
    return segments

In [76]:
results = get_segments('s01_e07', data)
for x in results:
    print(x)

Central Perk is proud to present
the music of Miss Phoebe Buffay
I want to start with a
song thats about that moment when
you suddenly realize what life is
The entire city is blacked out
Mom says it's all of Manhattan
and they have no idea when
Who am I gonna meet in
Could we talk about this later
I want to call my apartment
I am trapped in an ATM
that is the part to focus
I'm just stuck at the bank
I saw you with some guy
and these are the only candles
Ugly Naked Guy lit a bunch
It's been fourteen and a half
minutes and you still have not
Would you like to call somebody
about 300 guys I went to
I'm trppd in an ATM vstbl
I have no idea what you
He's trapped in an ATM vestibule
like that thought never entered my
my weirdest place would have to
the women's room on the second
floor of the New York CIty
What were you doing in a
'It's a Small World After All
Carol and I went behind a
couple of those mechanical Dutch children
and we were asked never to
The weirdest place would have to
I just

In [78]:
episode_keys = tuple(data.keys())
print(episode_keys)

('s01_e01', 's01_e02', 's01_e03', 's01_e04', 's01_e05', 's01_e06', 's01_e07', 's01_e08', 's01_e09', 's01_e10', 's01_e11', 's01_e12', 's01_e13', 's01_e14', 's01_e15', 's01_e16', 's01_e17', 's01_e18', 's01_e19', 's01_e20', 's01_e21', 's01_e22', 's01_e23', 's01_e24', 's06_e01', 's06_e02', 's06_e03', 's06_e04', 's06_e05', 's06_e06', 's06_e07', 's06_e08', 's06_e09', 's06_e10', 's06_e11', 's06_e12', 's06_e13', 's06_e14', 's06_e15', 's06_e16', 's06_e17', 's06_e18', 's06_e19', 's06_e20', 's06_e21', 's06_e22', 's06_e23', 's06_e24', 's06_e25', 's10_e01', 's10_e02', 's10_e03', 's10_e04', 's10_e05', 's10_e06', 's10_e07', 's10_e08', 's10_e09', 's10_e10', 's10_e11', 's10_e12', 's10_e13', 's10_e14', 's10_e15', 's10_e16', 's10_e17', 's10_e18', 's07_e01', 's07_e02', 's07_e03', 's07_e04', 's07_e05', 's07_e06', 's07_e07', 's07_e08', 's07_e09', 's07_e10', 's07_e11', 's07_e12', 's07_e13', 's07_e14', 's07_e15', 's07_e16', 's07_e17', 's07_e18', 's07_e19', 's07_e20', 's07_e21', 's07_e22', 's07_e23', 's07_e24'

In [79]:
episode_indexs = {}
for item in tqdm(episode_keys[:2]):
    temp = []
    segments = get_segments(item, data)
    for segment in tqdm(segments):
        for i, subtitle in enumerate(en_subtitle):
            if segment in subtitle:
                temp.append([i, segment, en_subtitle[i], zh_subtitle[i]])
    episode_indexs[item] = temp
pkl.dump(episode_indexs, open('episode_indexs.pkl', 'wb'))

100%|██████████| 2/2 [08:22<00:00, 251.39s/it]


In [11]:
with open('episode_indexs_10_30.pkl', 'rb') as f:
    results = pkl.load(f)

In [12]:
# Look alignment for each episode
for item in results:
    # print(item, len(results[item]))
    for x in results[item]:
        # print(item, len(results[item]))
        print(x[1], len(x[1].strip().split(' ')))
        # if len(results[item]) < 6:
        #     print(item, len(results[item]))
        #     print(x)
    print('=='*50)

Do you think they have yesterday's 6
our hearts and plunge us into 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I can't believe you did that 6
I

In [13]:
print(len(episode_keys))

NameError: name 'episode_keys' is not defined

In [16]:
# Check the alignment using indexs
target_index_dict = {}

for item in results:
    print(item)
    dic = {}
    for x in results[item]:
        dic[x[0]] = x[1:]
    idxs = sorted(list(dic.keys()))
    print(idxs)
    gaps = []
    for i in range(len(idxs)-1):
        gaps.append(idxs[i+1]-idxs[i])
    print(gaps)
    for i in range(len(gaps)-3):
        if max(gaps[i: i+3]) < 1000:
            print("Target Indexs:", idxs[i: i+3])
            target_index_dict[item] = idxs[i: i+3]
            break
    print('=='*50)

s01_e11
[19660, 31117, 78636, 125454, 148399, 186728, 203235, 281174, 309187, 334520, 416315, 470280, 543547, 564519, 647460, 677705, 733694, 737567, 752654, 806868, 820516, 839862, 851932, 899045, 938779, 948933, 964189, 964194, 970052, 1018027, 1019910, 1020589, 1068885, 1080493, 1109140, 1223097, 1247236, 1253633, 1299854, 1300095, 1346277, 1364709, 1444278, 1485287, 1521653, 1525208, 1525753, 1547154, 1551070, 1569297, 1593856, 1621717, 1626170, 1627873, 1646534, 1650825, 1666090, 1740404, 1747767, 1750223, 1751675, 1772002, 1795937, 1795940, 1795950, 1795971, 1795974, 1795976, 1795988, 1796001, 1796011, 1796017, 1796019, 1796024, 1796025, 1796026, 1796037, 1796039, 1796053, 1796068, 1796071, 1796072, 1796078, 1796079, 1796096, 1796099, 1796100, 1796111, 1796114, 1796115, 1796118, 1796122, 1796126, 1796129, 1796135, 1796141, 1796154, 1796157, 1796158, 1796163, 1796179, 1796180, 1796181, 1796183, 1796184, 1796193, 1796196, 1796203, 1796219, 1796232, 1796239, 1796240, 1796252, 179626

In [17]:
for item in target_index_dict:
    print(item, target_index_dict[item])

s01_e11 [1795937, 1795940, 1795950]
s01_e12 [901659, 902207, 902213]
s01_e13 [1797678, 1797683, 1797684]
s01_e14 [1798014, 1798037, 1798041]
s01_e15 [1799646, 1799651, 1799657]
s01_e16 [1796313, 1796317, 1796338]
s01_e17 [1106482, 1106720, 1106792]
s01_e18 [1795612, 1795627, 1795630]
s01_e19 [1795254, 1795263, 1795280]
s01_e20 [1798670, 1798674, 1798676]
s01_e21 [1799029, 1799033, 1799037]
s01_e22 [1799307, 1799323, 1799335]
s01_e23 [1797329, 1797333, 1797336]
s01_e24 [1794925, 1794931, 1794933]
s06_e03 [5119405, 5119410, 5119485]
s06_e04 [1106482, 1106720, 1106792]


In [18]:
season_dict = {}
for item in target_index_dict:
    temp = item.strip().split('_')
    season_key = temp[0]
    episode_key = temp[1]
    if season_key not in season_dict:
        season_dict[season_key] = {}
    else:
        season_dict[season_key][episode_key] = target_index_dict[item]
            # .append({episode_key: target_index_dict[item]})
print(season_dict)

{'s01': {'e12': [901659, 902207, 902213], 'e13': [1797678, 1797683, 1797684], 'e14': [1798014, 1798037, 1798041], 'e15': [1799646, 1799651, 1799657], 'e16': [1796313, 1796317, 1796338], 'e17': [1106482, 1106720, 1106792], 'e18': [1795612, 1795627, 1795630], 'e19': [1795254, 1795263, 1795280], 'e20': [1798670, 1798674, 1798676], 'e21': [1799029, 1799033, 1799037], 'e22': [1799307, 1799323, 1799335], 'e23': [1797329, 1797333, 1797336], 'e24': [1794925, 1794931, 1794933]}, 's06': {'e04': [1106482, 1106720, 1106792]}}


In [20]:
for x in season_dict:
    print(x, season_dict[x])
    print('=='*50)

s01 {'e12': [901659, 902207, 902213], 'e13': [1797678, 1797683, 1797684], 'e14': [1798014, 1798037, 1798041], 'e15': [1799646, 1799651, 1799657], 'e16': [1796313, 1796317, 1796338], 'e17': [1106482, 1106720, 1106792], 'e18': [1795612, 1795627, 1795630], 'e19': [1795254, 1795263, 1795280], 'e20': [1798670, 1798674, 1798676], 'e21': [1799029, 1799033, 1799037], 'e22': [1799307, 1799323, 1799335], 'e23': [1797329, 1797333, 1797336], 'e24': [1794925, 1794931, 1794933]}
s06 {'e04': [1106482, 1106720, 1106792]}


In [10]:
for item in sorted(season_dict.keys()):
    episodes = []
    print(item)
    for x in sorted(season_dict[item].keys()):
        print(x, season_dict[item][x])
    # print(sorted(season_dict[item].keys()))
    # print(season_dict[item])
    print("=="*50)

s01
e02 [1667529, 1667534, 1667537]
e03 [1667911, 1667920, 1667921]
e04 [1665715, 1665719, 1665720]
e05 [1666807, 1666813, 1666824]
e06 [1666435, 1666439, 1666452]
e07 [1666090, 1666091, 1666118]
e08 [1665047, 1665065, 1665068]
e09 [1665367, 1665384, 1665410]
e10 [1667167, 1667184, 1667188]
